In [16]:
#First part
import os
import json
import codecs
#Get all the files in the directory
data_path = 'data/2015-03-18'
files = os.listdir(data_path)
names = []
titles = []
#Parse through the files
for filename in files:
    file_path = data_path + '/' + filename
    with codecs.open(file_path,'r', encoding = 'utf-8') as file:
        prev_line = ''
        for line in file:
            if '<title>' in prev_line:
                titleline = line.lstrip()
                for i in range(len(titleline)):
                    c = titleline[i]
                    if c == ':':
                        break
                name = titleline[:i]
                title = titleline[i+2:-1]
                break
            prev_line = line
        names.append(name)
        titles.append(title)

names = json.dumps(names,ensure_ascii = 'False')
titles = json.dumps(titles)

output = [{'artist': artist, 'works': title} for artist, title in zip(names,titles)]



["Rembrandt Harmensz. van Rijn", "Louis Marcoussis", "Joan Mir\u00f3", "Pablo Picasso", "Marc Chagall"]
["Christ at Emmaus: The smaller Plate", "La Table", "Femme et Chien devant la Lune", "Quatre Femmes nues et T\u00eate sculpt\u00e9e, from: La Suite Vollard", "Self-Portrait"]
